# Managing custom Jupyter environments with Singularity
---
In this tutorial we will cover the basic workflow for managing custom software environments for Jupyter Notebooks using Singularity.

## Pull the base image from SingularityHub
We maintain a base image on SingularityHub for running Jupyter* (https://singularity-hub.org/collections/440/). This image contains the minimum dependencies and configuration needed to run containerized Notebooks (standalone or JupyterHub-spawned), and is intended to serve as a base for user-built software environments.

### A basic pull
Start by pulling the Jupyter base image from SingularityHub:

In [7]:
singularity pull --name "jupyter.img" shub://ResearchComputing/singularity-jupyter:master

Progress |===================================| 100.0% 


Done. Container is at: ./jupyter.img


There it is! Your container is good to go.

In [8]:
singularity exec jupyter.img jupyter -h

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the jupyter command's version and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json

Available subcommands: bundlerextension kernelspec migrate nbconvert
nbextension notebook run serverextension troubleshoot trust


## Customizing the base image
The base image we provide is meant to capture the _minimum_ config and dependencies to run Jupyter Notebooks. Here we detail how to customize the base image to better suit your needs.

### Resize the image
This image uses the default size set by Singularity _(768M)_ which is great for quick builds and pulls, but it is likely you'll need more space to accommodate your custom software stack.

In [9]:
ls -lsah | grep jupyter.img

769M -rwxrwxr-x.  1 vagrant vagrant 769M Sep 27 02:55 jupyter.img


To increase the size of your image you must create a new image, and then import the Jupyter base image:

In [13]:
singularity create --size 1500 jupyter-ext.img
singularity export jupyter.img | singularity import jupyter-ext.img

Initializing Singularity image subsystem
Opening image file: jupyter-ext.img
Creating 1500MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-ext.img


In [14]:
singularity exec jupyter.img jupyter -h

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the jupyter command's version and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json

Available subcommands: bundlerextension kernelspec migrate nbconvert
nbextension notebook run serverextension troubleshoot trust


### Installing software _(the quick way)_
By default Singularity containers mounted as read-only volumes, which means you won't be able to add content or install software _(even as a privileged user)_, save for default or system-mounted paths. In order to add content you must run your Singularity command with the `--writable` flag.

In [ ]:
singularity shell --writable jupyter-ext.img

Singularity: Invoking an interactive shell within container...



### Installing Software _(the reproducible way)_
Shelling into your container and making ad-hoc changes is excellent for debugging and initial development, but it is considered bad practice as the steps needed to construct your software environment are not captured and cannot be reproduced.

To make durable, reproducible changes you need to build a spec file from which you can bootstrap your container. Bootstrapping must be done by a privileged user

In [4]:
singularity create --size 1500 jupyter-bootstrapped.img
sudo /usr/local/bin/singularity bootstrap jupyter-bootstrapped.img jupyter-bootstrapped.def

Initializing Singularity image subsystem
Opening image file: jupyter-bootstrapped.img
Creating 1500MiB image
Binding image to loop
Creating file system within image
Image is done: jupyter-bootstrapped.img
Sanitizing environment
Building from bootstrap definition recipe
Adding base Singularity environment to container
Docker image path: index.docker.io/jupyter/base-notebook:latest
Cache folder set to /root/.singularity/docker
Exploding layer: sha256:e0a742c2abfd5e2a6f8ed15b1c78e873cf9559b96a04204daf6de5df01e3124c.tar.gz
Exploding layer: sha256:486cb8339a27635fa93dc47aa0c689326a0a7cce388966d16daf8d265436cf7f.tar.gz
Exploding layer: sha256:dc6f0d824617ad8a5d1163a5b2084814665dd83156317ad06ccf14deb517a053.tar.gz
Exploding layer: sha256:4f7a5649a30e3f318ce5d7e4dbcbbeb6c0938c4cbae4d4a641fe910562ff4978.tar.gz
Exploding layer: sha256:672363445ad2c734e29221a6b47f4e614b5adc8a3cdca3364f62db2ed2bdff0c.tar.gz
Exploding layer: sha256:b337aaee648d9f87e96fae8b24ae2dd887a2ded309b38dbee691fcdb040878cc.ta

: 255

In [3]:
rm jupyter-bootstrapped.img

# Docker, since shub bootstrap unsupported

In [ ]:
singularity pull --name "jupyter-base.img" docker://jupyter/base-notebook
singularity create --size 1500 jupyter-ext.img
singularity export jupyter-base.img | singularity import jupyter-ext.img

```
BootStrap: docker
From: jupyter/base-notebook

%post
  echo "Installing seaborn..."
  pip3 install seaborn
```

In [1]:
ls

demo-playbook		  LICENSE
hello-from-r.ipynb	  managing-custom-jupyter-envs-with-singularity.ipynb
jupyter-base.img	  README.md
jupyter-bootstrapped.def  singularity-spawner-and-docker.ipynb
jupyter-r.img		  Vagrantfile
